In [21]:
import re
import xml.etree.ElementTree as ET

xmlfile = open('vysledky.xml',encoding='iso-8859-2')
xmlfile = xmlfile.read()

#namespace pryc kvuli ElementTree
xmlstring = re.sub(' xmlns="[^"]+"', '', xmlfile, count=1)
root = ET.fromstring(xmlstring)

In [22]:
# XML parsing
vysledky = []

# skrutinium
hranice = 5 # kolik procent je potreba

strany = []
for strana in root.find("CR").iter("STRANA"):
    if float(strana.find("HLASY_STRANA").get("PROC_HLASU")) > hranice:
        strany.append(strana.get("ESTRANA"))

for kraj in root.iter("KRAJ"): 
        nazev = kraj.get("NAZ_KRAJ")
        hlasy = kraj.find("UCAST").get("PLATNE_HLASY")
        
        vyslstrany = []
        for strana in kraj.iter("HLASY_STRANA"):
            if strana.get("ESTRANA") in strany:
                vyslstrany.append([strana.get("ESTRANA"),int(strana.get("HLASY")),0])
                
        vysledky.append([nazev, [int(hlasy)], vyslstrany])
print(vysledky)

[['Hlavní město Praha', [345558], [['5', 65259, 0], ['9', 13156, 0], ['26', 71687, 0], ['27', 66099, 0], ['28', 19958, 0], ['30', 46137, 0], ['39', 16505, 0]]], ['Středočeský kraj', [311493], [['5', 53828, 0], ['9', 19929, 0], ['26', 44125, 0], ['27', 47519, 0], ['28', 24951, 0], ['30', 60881, 0], ['39', 11680, 0]]], ['Jihočeský kraj', [147224], [['5', 21266, 0], ['9', 11482, 0], ['26', 16709, 0], ['27', 20214, 0], ['28', 13660, 0], ['30', 31694, 0], ['39', 9764, 0]]], ['Plzeňský kraj', [125754], [['5', 19605, 0], ['9', 10218, 0], ['26', 13887, 0], ['27', 15645, 0], ['28', 12067, 0], ['30', 28899, 0], ['39', 5292, 0]]], ['Karlovarský kraj', [53535], [['5', 6136, 0], ['9', 3934, 0], ['26', 5233, 0], ['27', 6599, 0], ['28', 6447, 0], ['30', 14478, 0], ['39', 1402, 0]]], ['Ústecký kraj', [141720], [['5', 17696, 0], ['9', 13008, 0], ['26', 11747, 0], ['27', 16363, 0], ['28', 15786, 0], ['30', 40738, 0], ['39', 2938, 0]]], ['Liberecký kraj', [97800], [['5', 13473, 0], ['9', 5436, 0], ['26',

In [23]:
# republikove mandatove cislo
hlasy_celkem = 0
for kraj in vysledky:
    hlasy_celkem += kraj[1][0]
rmc = round(hlasy_celkem/200)

In [24]:
# vypocet zakladniho poctu mandatu v jednotlivych krajich
mtotal = 0
for kraj in vysledky:
    mbase = kraj[1][0]/rmc
    kraj[1].extend(divmod(mbase,1))
    mtotal += int(mbase)

In [25]:
# dopocet do 200 mandatu
mmissing = 200 - mtotal

for kraj in sorted(vysledky, key=lambda kraj: kraj[1][2], reverse=True)[0:mmissing]:
    kraj[1][1] += 1

In [26]:
# dhondtuv prepocet

for kraj in vysledky:
    
    # deli se jednickou az poctem mandatu
    mandaty = int(kraj[1][1])
    
    dhondtlist = []
    
    # vypocteni podilu
    for strana in kraj[2]:    
        for i in range(mandaty):
            dhondtlist.append([strana[0], round(strana[1]/(i+1))]) 

    # rozdeleni mandatu
    for mandat in sorted(dhondtlist, key=lambda zaznam: zaznam[1], reverse=True)[0:mandaty]:
        for strana in kraj[2]:
            if mandat[0] == strana[0]:
                strana[2] += 1

In [27]:
# soucet

soucet = {}
for kraj in vysledky:
    for strana in kraj[2]:
        if strana [0] in soucet:
            soucet[strana[0]] += strana[2]
        else:
            soucet[strana[0]] = strana[2]
    
for zaznam in sorted(soucet.items(), key=lambda zaznam: zaznam[1], reverse=True):
    print(zaznam[0],":::",zaznam[1])

30 ::: 56
27 ::: 36
5 ::: 35
26 ::: 25
28 ::: 19
9 ::: 15
39 ::: 14
